### Package dependencies

## Document Ingestion Pipeline

In [ ]:
### ID of SME for data labeling

sme_id = 'sme_001'

In [ ]:
import os
import openai
import time

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer, StorageContext
from llama_index.core import DocumentSummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

#### Parsing files and loading documents

In [ ]:
data_root = "./ritika_data/"

In [ ]:
## Ingesting documents from Local

filenames = ["Decarbonizing-the-Built-World-A-Call-to-Action-2023-03-07",
             "Digital-Twin-Capabilities-Periodic-Table-User-Guide",
             "Digital-Twin-System-Interoperability-Framework-12072021",
             "DTC-Reality-Capture-Industry-User-Guide-for-Tenant-Improvement-Projects-2023-06-07",
             "Infrastructure-Digital-Twin-Maturity-Model"]
             #"Platform-Stack-Architectural-Framework",
             #"Reality-Capture-A-Digital-Twin-Foundation",
             #"SMM-Digital-Twin-Profile-2022-06-20",
             #"User-Guide-1-Why-and-What-2023-07-18",
             #"User-Guide-2-Identifying-and-Aligning",
             #"User-Guide-3_A-Whole_Systems_Approach"]

docs = []

for filename in filenames:
    doc = SimpleDirectoryReader(input_files=[f"{data_root}/{filename}.pdf"]).load_data()
    doc[0].doc_id = filename.replace(".pdf","")
    docs.extend(doc)

#### Redis-Based Ingestion Pipeline for Doc store and Vector store

In [ ]:
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.storage.index_store.redis import RedisIndexStore

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0125", temperature=0)
text_splitter = SentenceSplitter(chunk_size=3000)
embed_model = OpenAI(model="text-embedding-3-small")

#### Setting up document store, index store and vector store

In [ ]:
## Setting up connection to Redis two ways:

## Through Python Redis client:

import redis
redis_client = redis.Redis(
  host='',
  port=,
  password='')

## Through host and port:

#REDIS_HOST = os.getenv("REDIS_HOST", "127.0.0.1")
#REDIS_PORT = os.getenv("REDIS_PORT", 6379)

In [ ]:
## Defining PGVector database:

from sqlalchemy import make_url
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

url = make_url("postgres://postgres.<username>:<password>@aws-0-us-west-1.pooler.supabase.com:5432/postgres")
db_name="postgres"
vector_store = PGVectorStore.from_params(
    database=url.database,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=sme_id,
    embed_dim=1536,  # openai embedding dimension
)

#### Creating storage context

In [ ]:
## Through Redis client:

from llama_index.core import SimpleDirectoryReader

storage_context = StorageContext.from_defaults(
    docstore=RedisDocumentStore.from_redis_client(
        redis_client=redis_client, namespace=sme_id
    ),
    
    index_store=RedisIndexStore.from_redis_client(
        redis_client=redis_client, namespace=sme_id
    ),
    vector_store = vector_store
)

response_synthesizer = get_response_synthesizer(
    response_mode = "tree_summarize", use_async=True
)

In [ ]:
## Through Redis host and port:

storage_context = StorageContext.from_defaults(
    docstore=RedisDocumentStore.from_redis_client(
        host=REDIS_HOST, port=REDIS_PORT, namespace="llama_index"
    ),
    
    index_store=RedisIndexStore.from_redis_client(
        host=REDIS_HOST, port=REDIS_PORT, namespace="llama_index"
    ),
    vector_store = vector_store
)

response_synthesizer = get_response_synthesizer(
    response_mode = "tree_summarize", use_async=True
)

#### Creating Document Summary Index and storing in respective db locations

In [ ]:
doc_summary_index = DocumentSummaryIndex.from_documents(docs,
    llm=llm,
    transformations=[text_splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
    storage_context = storage_context)

In [ ]:
### Fin ###

In [ ]:
### Restart kernel to test Pipeline below:

## Document Retreival Pipeline

In [1]:
### ID of SME for data labeling

sme_id = ''

In [3]:
import os
import openai
import time

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer, StorageContext
from llama_index.core import DocumentSummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

#### Recreating storage context object using same DB connections from ingestion pipeline

In [6]:
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.storage.index_store.redis import RedisIndexStore

In [7]:
llm = OpenAI(model="gpt-3.5-turbo-0125", temperature=0)
text_splitter = SentenceSplitter(chunk_size=3000)
embed_model = OpenAI(model="text-embedding-3-small")

In [9]:
## Connecting to Redis DB through Python Redis Client (Refer to ingestion pipeline above for Redis Host-Port access)

import redis
redis_client = redis.Redis(
  host='',
  port=,
  password='')

## Setting up PGVector database

from sqlalchemy import make_url
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

url = make_url("postgres://postgres.<username>:<password>@aws-0-us-west-1.pooler.supabase.com:5432/postgres")
db_name="postgres"
vector_store = PGVectorStore.from_params(
    database=url.database,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=sme_id,
    embed_dim=1536,  # openai embedding dimension
)


In [10]:
## Re-creating Storage context through Redis client (Refer to ingestion pipeline above for Redis Host-Port access):

from llama_index.core import SimpleDirectoryReader

storage_context = StorageContext.from_defaults(
    docstore=RedisDocumentStore.from_redis_client(
        redis_client=redis_client, namespace=sme_id
    ),
    index_store=RedisIndexStore.from_redis_client(
        redis_client=redis_client, namespace=sme_id
    ),
    vector_store = vector_store
)

response_synthesizer = get_response_synthesizer(
    response_mode = "tree_summarize", use_async=True
)

#### Creating steps to final response:

In [11]:
## Creating prompt template

from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

new_summary_tmpl_str = (
    "Context information is below:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge,"
    "answer the query in the style of a McKinsey, Bain or BCG consultant who specializes in digital transformation strategies."
    "Your goal is to help business users succeed in their digital transformation journeys."
    "Do not use any context outside of these documents."
    "If a question is outside of your area of expertise, politely refuse to answer and suggest alternative topics of discussion from the context provided."
    "You should maintain a friendly yet professional tone."
    "Use detailed bullet points whenever relevant.\n"
    "Query: {query_str}\n"
    "Answer: "
)

new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

#### Loading document summary index using storage context:

In [12]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

from llama_index.core import load_index_from_storage

doc_summary_index = load_index_from_storage(
    storage_context=storage_context
)

# Configuring response synthesizer
response_synthesizer = get_response_synthesizer(streaming=True, response_mode="tree_summarize")

## Creating Retreiver object
retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    choice_batch_size=10,
    choice_top_k=5
)

# Assembling query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)

## Checking default query prompt:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

## Modifying query prompt

query_engine.update_prompts(
    {"response_synthesizer:summary_template": new_summary_tmpl}
)

## Checking modified query prompt:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:summary_template<br>**Text:** <br>

Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:summary_template<br>**Text:** <br>

Context information is below:
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,answer the query in the style of a McKinsey, Bain or BCG consultant who specializes in digital transformation strategies.Your goal is to help business users succeed in their digital transformation journeys.Do not use any context outside of these documents.If a question is outside of your area of expertise, politely refuse to answer and suggest alternative topics of discussion from the context provided.You should maintain a friendly yet professional tone.Use detailed bullet points whenever relevant.
Query: {query_str}
Answer: 


<br><br>

### Implementing embeddings Retirever

In [13]:
start_time = time.time()
# query
response = query_engine.query("What are the most important considerations when implementing a digital twin?")
response.print_response_stream()

## Use streaming response in block:
'''for text in streaming_response.response_gen:
    # do something with text as they arrive.
    pass
'''

print("--- %s seconds ---" % (time.time() - start_time))

When implementing a digital twin, it is crucial to consider the following key factors:

1. Holistic Approach: View the digital twin as a holistic system of systems that are interconnected within a common digital thread. This ensures that all components work together seamlessly to provide maximum value.

2. Specific Use Cases: Connect the digital twin to solving specific use cases relevant to your business. Without this connection, the digital twin may lack the ability to provide tangible value.

3. Data Communication and Collaboration: Ensure that the digital twin facilitates effective communication, collaboration, and correlation of data in a meaningful way. This will enable stakeholders to make informed decisions based on real-time insights.

4. Robust Tool Development: Develop a robust digital twin tool that adds value throughout the entire digital building lifecycle. This includes considering factors such as data accuracy, scalability, and integration with existing systems.

5. Sta

In [ ]:
### Fin ###

In [ ]:
### Restart kernel to test Pipeline below:

## SME Data Insertion-Deletion (updation) pipeline

In [ ]:
### ID of SME for data labeling

sme_id = 'sme_001'

In [ ]:
import os
import openai
import time

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer, StorageContext
from llama_index.core import DocumentSummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

#### Parsing new files

In [ ]:
data_root = "./ritika_data/"

In [ ]:
filenames = [#"Decarbonizing-the-Built-World-A-Call-to-Action-2023-03-07",
             #"Digital-Twin-Capabilities-Periodic-Table-User-Guide",
             #"Digital-Twin-System-Interoperability-Framework-12072021",
             #"DTC-Reality-Capture-Industry-User-Guide-for-Tenant-Improvement-Projects-2023-06-07",
             #"Infrastructure-Digital-Twin-Maturity-Model"]
             "Platform-Stack-Architectural-Framework",
             "Reality-Capture-A-Digital-Twin-Foundation",
             "SMM-Digital-Twin-Profile-2022-06-20",
             "User-Guide-1-Why-and-What-2023-07-18",
             "User-Guide-2-Identifying-and-Aligning",
             "User-Guide-3_A-Whole_Systems_Approach"]

new_docs = []

for filename in filenames:
    doc = SimpleDirectoryReader(input_files=[f"{data_root}/{filename}.pdf"]).load_data()
    doc[0].doc_id = filename.replace(".pdf","")
    new_docs.extend(doc)

#### Redis-Based Ingestion Pipeline for Doc store and Vector store

In [ ]:
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.storage.index_store.redis import RedisIndexStore

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0125", temperature=0)
text_splitter = SentenceSplitter(chunk_size=3000)
embed_model = OpenAI(model="text-embedding-3-small")

#### Setting up document store, index store and vector store

In [ ]:
## Setting up connection to Redis two ways:

## Through Python Redis client:

import redis
redis_client = redis.Redis(
  host='',
  port=,
  password='')

In [ ]:
## Defining PGVector database:

from sqlalchemy import make_url
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

url = make_url("postgres://postgres.<username>:<password>@aws-0-us-west-1.pooler.supabase.com:5432/postgres")
db_name="postgres"
vector_store = PGVectorStore.from_params(
    database=url.database,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=sme_id,
    embed_dim=1536,  # openai embedding dimension
)

#### Loading Index from storage

In [ ]:
## Re-creating Storage context through Redis client (Refer to ingestion pipeline above for Redis Host-Port access):

from llama_index.core import SimpleDirectoryReader

storage_context = StorageContext.from_defaults(
    docstore=RedisDocumentStore.from_redis_client(
        redis_client=redis_client, namespace=sme_id
    ),
    index_store=RedisIndexStore.from_redis_client(
        redis_client=redis_client, namespace=sme_id
    ),
    vector_store = vector_store
)

response_synthesizer = get_response_synthesizer(
    response_mode = "tree_summarize", use_async=True
)

In [ ]:
from llama_index.core import load_index_from_storage

doc_summary_index = load_index_from_storage(
    storage_context=storage_context
)

In [ ]:
doc_summary_index.refresh_ref_docs(new_docs)

### Checking response after adding new data

In [ ]:
## Creating prompt template

from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

new_summary_tmpl_str = (
    "Context information is below:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge,"
    "answer the query in the style of a McKinsey, Bain or BCG consultant who specializes in digital transformation strategies."
    "Your goal is to help business users succeed in their digital transformation journeys."
    "Do not use any context outside of these documents."
    "If a question is outside of your area of expertise, politely refuse to answer and suggest alternative topics of discussion from the context provided."
    "You should maintain a friendly yet professional tone."
    "Use detailed bullet points whenever relevant.\n"
    "Query: {query_str}\n"
    "Answer: "
)

new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

from llama_index.core import load_index_from_storage

#doc_summary_index = load_index_from_storage(
#    storage_context=storage_context
#)

# Configuring response synthesizer
response_synthesizer = get_response_synthesizer(streaming=True, response_mode="tree_summarize")

## Creating Retreiver object
retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    choice_batch_size=10,
    choice_top_k=5
)

# Assembling query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)

## Checking default query prompt:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

## Modifying query prompt

query_engine.update_prompts(
    {"response_synthesizer:summary_template": new_summary_tmpl}
)

## Checking modified query prompt:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

### Implementing embeddings Retirever

In [ ]:
start_time = time.time()
# query
response = query_engine.query("How can companies think about setting up their digital twins?")
response.print_response_stream()

## Use streaming response in block:
'''for text in streaming_response.response_gen:
    # do something with text as they arrive.
    pass
'''

print("--- %s seconds ---" % (time.time() - start_time))